In [3]:
import random as r
import time

from engine import Value
from modules import *
from ops import *
from gpt import *

In [4]:
# load the dataset
with open('../input.txt', 'r', encoding='utf-8') as f:
    tinyShakespeare_string = f.read()

In [5]:
# Convert the string to bytes
unique_chars = set(tinyShakespeare_string)
print(unique_chars)
v = len(unique_chars)
encode_dict, decode_dict = {}, {}
for i, c in enumerate(unique_chars):
    encode_dict[c] = i
    decode_dict[i] = c
print(encode_dict)
print(decode_dict)

{'d', 'm', 'O', 'h', 'l', 'o', 'S', 'C', 'w', 'V', 'k', 'B', 't', '?', '3', 'v', 'A', 'Q', 'X', 'D', 'x', 'E', 'P', 'T', "'", '!', 'i', 'q', 'G', '$', 'R', 'U', 'F', 'Z', ',', ';', 'M', 'p', 'u', 'N', ':', 'L', 'I', 'g', '.', '-', 'f', '&', 's', 'n', 'j', 'r', 'e', 'H', 'K', 'J', 'Y', '\n', ' ', 'z', 'c', 'y', 'a', 'W', 'b'}
{'d': 0, 'm': 1, 'O': 2, 'h': 3, 'l': 4, 'o': 5, 'S': 6, 'C': 7, 'w': 8, 'V': 9, 'k': 10, 'B': 11, 't': 12, '?': 13, '3': 14, 'v': 15, 'A': 16, 'Q': 17, 'X': 18, 'D': 19, 'x': 20, 'E': 21, 'P': 22, 'T': 23, "'": 24, '!': 25, 'i': 26, 'q': 27, 'G': 28, '$': 29, 'R': 30, 'U': 31, 'F': 32, 'Z': 33, ',': 34, ';': 35, 'M': 36, 'p': 37, 'u': 38, 'N': 39, ':': 40, 'L': 41, 'I': 42, 'g': 43, '.': 44, '-': 45, 'f': 46, '&': 47, 's': 48, 'n': 49, 'j': 50, 'r': 51, 'e': 52, 'H': 53, 'K': 54, 'J': 55, 'Y': 56, '\n': 57, ' ': 58, 'z': 59, 'c': 60, 'y': 61, 'a': 62, 'W': 63, 'b': 64}
{0: 'd', 1: 'm', 2: 'O', 3: 'h', 4: 'l', 5: 'o', 6: 'S', 7: 'C', 8: 'w', 9: 'V', 10: 'k', 11: 'B

In [9]:
tinyShakespeare_chars = [encode_dict[c] for c in tinyShakespeare_string]
print(tinyShakespeare_string[:100])
print(tinyShakespeare_chars[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
[32, 26, 51, 48, 12, 58, 7, 26, 12, 26, 59, 52, 49, 40, 57, 11, 52, 46, 5, 51, 52, 58, 8, 52, 58, 37, 51, 5, 60, 52, 52, 0, 58, 62, 49, 61, 58, 46, 38, 51, 12, 3, 52, 51, 34, 58, 3, 52, 62, 51, 58, 1, 52, 58, 48, 37, 52, 62, 10, 44, 57, 57, 16, 4, 4, 40, 57, 6, 37, 52, 62, 10, 34, 58, 48, 37, 52, 62, 10, 44, 57, 57, 32, 26, 51, 48, 12, 58, 7, 26, 12, 26, 59, 52, 49, 40, 57, 56, 5, 38]


In [11]:
split_size = int(0.95 * len(tinyShakespeare_chars))
train_dataset, val_dataset = tinyShakespeare_chars[:split_size], tinyShakespeare_chars[split_size:]
train_pointer, val_pointer = 0, 0
def get_batch(batch_size, seq_len, val = False):
    global train_pointer, val_pointer
    tok_ct = batch_size * seq_len
    dataset = val_dataset if val else train_dataset
    pointer = val_pointer if val else train_pointer
    input_toks = dataset[pointer:pointer + tok_ct] # grabbing sequential data is bad practice but whatever
    target_toks = dataset[pointer + 1:pointer + tok_ct + 1]
    if val:
        val_pointer += tok_ct
    else:
        train_pointer += tok_ct
    input_toks = split_dim(input_toks, (batch_size, seq_len))
    target_toks = split_dim(target_toks, (batch_size, seq_len))
    return input_toks, target_toks
input_toks, target_toks = get_batch(2, 10)
print(get_shape(input_toks), get_shape(target_toks))
pretty_tensor_print(input_toks)
pretty_tensor_print(target_toks)

[2, 10] [2, 10]
[
  [32, 26, 51, 48, 12, 58, 7, 26, 12, 26]
  [59, 52, 49, 40, 57, 11, 52, 46, 5, 51]
]
[
  [26, 51, 48, 12, 58, 7, 26, 12, 26, 59]
  [52, 49, 40, 57, 11, 52, 46, 5, 51, 52]
]


In [13]:
config = {
    'vocab_len':v,
    'model_dim':4,
    'max_seq_len':20,
    'num_heads':2,
    'head_dim':2,
    'mlp_mult':2,
    'dropout_rate':0.1,
    'num_layers':1
}
model = GPT(config)

eta = 0.01

batch_size = 8
toks_per_batch = batch_size * config['max_seq_len']
train_iterations = min(split_size // toks_per_batch, 1000)
val_iterations = min((len(tinyShakespeare_chars) - split_size) // toks_per_batch, 50)
val_frequency = train_iterations // val_iterations
print(train_iterations, val_iterations, val_frequency)

1000 50 20


In [15]:
def greedy_inference(model, input, gen_len):
    gen_len = min(gen_len, config['max_seq_len'] - len(input) - 1)
    toks = [[encode_dict[c] for c in input]]
    for i in range(gen_len):
        probabilities, _ = model(toks)
        argmax = float('-inf')
        argmax_idx = None
        for i, val in enumerate(probabilities[0][-1]):
            if val.data > argmax:
                argmax_idx = i
                argmax = val.data
        toks[0].append(argmax_idx)
    return "".join(decode_dict[t] for t in toks[0])
print(greedy_inference(model, "King Ri", 10))

King RiLhNhoNNhhh


In [ ]:
pointer = 0
start_time = time.time()
for i in range(train_iterations):
    # forward pass
    train_input_toks, train_target_toks = get_batch(batch_size, config['max_seq_len'])
    probabilities, train_loss = model(train_input_toks, train_target_toks)
        
    if i % val_frequency == 0:
        val_input_toks, val_target_toks = get_batch(batch_size, config['max_seq_len'], val = True)
        probabilities, val_loss = model(val_input_toks, val_target_toks)
        
        print(f'step {i} ({(i / train_iterations):.1f}%) | train loss: {train_loss.data:.2f} | val loss: {val_loss.data:.2f} | ' 
              f'time: {int(time.time() - start_time)}sec | example: {greedy_inference(model, "King Ri", 10)}')

    ## backward pass
    #set param gradients to 0
    for p in model.parameters():
        p.grad = 0.0
    # clac gradients
    train_loss.backward()
    # performing a step of SGD
    for p in model.parameters():
        p.data -= eta * p.grad

step 0 (0.0%) | train loss: 667.90 | val loss: 667.90 | time: 5sec | example: King RiLhNhoNNhhh
step 20 (0.0%) | train loss: 545.34 | val loss: 553.74 | time: 100sec | example: King Ri          
step 40 (0.0%) | train loss: 535.66 | val loss: 516.84 | time: 199sec | example: King Ri          
step 60 (0.1%) | train loss: 529.96 | val loss: 527.38 | time: 297sec | example: King Ri          
step 80 (0.1%) | train loss: 555.99 | val loss: 546.65 | time: 397sec | example: King Ri          
step 100 (0.1%) | train loss: 514.19 | val loss: 516.15 | time: 511sec | example: King Ri          
step 120 (0.1%) | train loss: 524.04 | val loss: 599.85 | time: 609sec | example: King Ri          
step 140 (0.1%) | train loss: 495.97 | val loss: 545.58 | time: 709sec | example: King Ri          
step 160 (0.2%) | train loss: 515.41 | val loss: 578.22 | time: 885sec | example: King Ri          
step 180 (0.2%) | train loss: 559.99 | val loss: 571.34 | time: 996sec | example: King Ri          


In [22]:
greedy_inference(model, "King Ri", 10)

'King Ri t t t e t'